In [1]:
# xmlrpc_client.ipynb

import xmlrpc.client as rpc
import pickle

server_proxy = rpc.ServerProxy("http://localhost:8010")

print('Ping:', server_proxy.exec('ping'))

print('Server datetime:', server_proxy.exec('now'))

print('View, type, value:', server_proxy.exec('type', 2))
print('View, type, value:', server_proxy.exec('type', 2.))
print('View, type, value:', server_proxy.exec('type', 'My string'))
print('View, type, value:', server_proxy.exec('type', "My string"))
print('View, type, value:', server_proxy.exec('type', [1, 2, 3]))
print('View, type, value:', server_proxy.exec('type', ["one", "two", "three"]))
print('View, type, value:', server_proxy.exec('type', (1, 2, "3")))

print('Sum 2 + 3 :', server_proxy.exec('sum', 2, 3))
print('Pow 2^3: ', server_proxy.exec('pow', 2, 3))

ConnectionRefusedError: [WinError 10061] Подключение не установлено, т.к. конечный компьютер отверг запрос на подключение

In [ ]:
from PIL import Image # Работа с изображением
from pylab import * # Отображение

import xmlrpc.client as rpc

# Режим отображения внутри ноутбука
%matplotlib inline

img = Image.open('../../resources/lr1/Jellyfish.jpg')

img_arr = array(img)

# Исходное изображение
imshow(img_arr)

In [ ]:
# Тест бинарной передачи данных
pimg = img_arr.dumps()
pimg = pickle.dumps(img_arr) # универсально

img_bin = rpc.Binary(pimg)

img_bin2 = server_proxy.exec('send_back_binary', img_bin)

#img_arr2 = np.loads(img_bin2.data)
img_arr2 = pickle.loads(img_bin2.data) # универсально

# Изображение после возрата с сервера
imshow(img_arr2)

In [ ]:
# Инверсия цвета изображения через сервер
def inv_color(img_arr_in):
    pimg = pickle.dumps(img_arr_in)
    img_bin = rpc.Binary(pimg)
    
    img_bin2 = server_proxy.exec('color_inversion', img_bin)
    
    img_arr_out = pickle.loads(img_bin2.data)
    return img_arr_out

img_arr_inv = inv_color(img_arr)

# Изображение с инверсией цвета
imshow(img_arr_inv)

In [ ]:
print(server_proxy.exec('black_list_check', 'Петров'))
print(server_proxy.exec('black_list_check', 'Примеров'))

In [ ]:
# Лабораторная 1. Задание 4. Проверка на наличие человека в черном списке с проверкой по ФИО и дате рождения

print(server_proxy.exec('black_list_check_by_name_and_birthday', 'Иванов Иван Иванович', '22.03.1989'))
print(server_proxy.exec('black_list_check_by_name_and_birthday', 'Иванов Иван Иванович', '23.03.1989'))

In [ ]:
colored_img = '../../resources/lr1/Jellyfish.jpg'
monochrome_img = '../../resources/lr1/11.bmp'

def show_modified_image(path, modify_func, param=None):
    img = Image.open(path)
    img_arr = array(img)
    pimg = pickle.dumps(img_arr)
    img_bin = rpc.Binary(pimg)
    if param is None:
        img_bin_out = server_proxy.exec(modify_func, img_bin)
    else:
        img_bin_out = server_proxy.exec(modify_func, img_bin, param)
    img_arr_out = pickle.loads(img_bin_out.data) 
    gray()
    imshow(img_arr_out)

# Лабораторная 1. Задание 5. Модифицировать функцию сервера send_back_inversion на корректную работу также с монохромными изображениями,
# т.е. входными данными могут быть размерности (M, N, 3) и (M, N, 1).

show_modified_image(colored_img, 'color_inversion')

In [ ]:
show_modified_image(monochrome_img, 'color_inversion')

In [ ]:
# Лабораторная 1. Задание 6. Функция бинаризации изображения по задаваемому порогу 1-255 для цветных и монохромных изображений

show_modified_image(colored_img, 'binarize_by_threshold', 100)

In [ ]:
show_modified_image(colored_img, 'binarize_by_threshold', 10)

In [ ]:
show_modified_image(monochrome_img, 'binarize_by_threshold', 100)

In [ ]:
# Лабораторная 1. Задание 7. Функция разворота изображения относительно вертикали для цветных и монохромных изображений

show_modified_image(colored_img, 'mirror_vertically')

In [ ]:
show_modified_image(monochrome_img, 'mirror_vertically')

In [ ]:
# Лабораторная 2. Задание 4. Получение содержимого журнала событий сервера статистики
# с возможностью получения среза по типу выполняемой операции (события) и времени

import datetime

all_logs = server_proxy.get_logs_slice('all', '', '', '', 1)
sliced_logs = server_proxy.get_logs_slice('binarize_by_threshold', '2023-09-26 19:18:33', '', 10, 1)
print('All logs:\n' + (all_logs if isinstance(all_logs, str) else '\n'.join(map(str, all_logs))))
print('\nSliced logs:\n' + (sliced_logs if isinstance(sliced_logs, str) else '\n'.join(map(str, sliced_logs))))

In [ ]:
# Лабораторная 3. Задание 5. Получение содержимого журнала событий сервера статистики с возможностью получения среза
# по типу выполняемой функции, времени вызова и ее длительности

import datetime

all_logs = server_proxy.get_logs_slice('all', '', '', '', 2)
sliced_logs = server_proxy.get_logs_slice('type', '2023-09-26 19:18:33', '', 10, 2)
print('All logs:\n' + (all_logs if isinstance(all_logs, str) else '\n'.join(map(str, all_logs))))
print('\nSliced logs:\n' + (sliced_logs if isinstance(sliced_logs, str) else '\n'.join(map(str, sliced_logs))))